# Amazon SQS (Simple Queue Service)

<img src="../_assets/aws_service_icons/sqs.svg" width="80" alt="Amazon SQS">

## Goals
- Understand what **SQS** is
- Know what it’s practically used for
- See a minimal **SDK pseudo-code** workflow (not executed)


## What is SQS?

**Amazon Simple Queue Service (SQS)** is a fully managed **message queue** on AWS.

It lets **producers** (services that create work) send messages to a **queue**, and **consumers** (services that do work) poll the queue and process those messages.

Key properties:
- **Decoupling**: producers and consumers do not need to know about each other.
- **Buffering**: absorbs spikes by queueing work until consumers can catch up.
- **Reliability**: messages are stored durably and can be retried.

SQS supports two queue types:
- **Standard**: very high throughput, **at-least-once** delivery, *best-effort* ordering.
- **FIFO**: preserves **ordering** within a message group and supports **exactly-once processing** semantics *within SQS* via deduplication (still design consumers to be idempotent).


## What is SQS used for (practically)?

SQS is commonly used whenever you want **asynchronous** communication and to **decouple** systems.

Typical use cases:
- **Background jobs** (image/video processing, report generation, email sending)
- **Microservice decoupling** (service A submits work; service B processes it)
- **Rate smoothing / burst handling** (queue buffers traffic spikes)
- **Event-driven pipelines** (e.g., S3 event → enqueue → worker processes)
- **Fan-out patterns** (SNS topic → multiple SQS queues → different consumers)
- **ML/DS workflows** (enqueue feature computation, inference jobs, or data validation tasks)


## Core concepts (minimum you should know)

- **Message**: a payload (up to 256 KB) + optional attributes.
- **Visibility timeout**: after a consumer receives a message, it becomes temporarily hidden so other consumers don’t process it at the same time.
- **Delete on success**: consumers should delete the message only after successful processing.
- **At-least-once delivery** (Standard): duplicates can happen → make processing **idempotent**.
- **Long polling**: reduce empty responses and cost by waiting for messages (`WaitTimeSeconds`).
- **Dead-letter queue (DLQ)**: messages that fail repeatedly can be moved to a separate queue for inspection/replay.
- **FIFO specifics**: ordered processing requires `MessageGroupId`; dedup uses `MessageDeduplicationId` (or content-based dedup).


## Using SQS with an SDK (pseudo-code)

This is **illustrative pseudo-code** showing the typical flow in an AWS SDK (e.g., Python `boto3`, JS v3, Java, etc.).

**Do not run as-is**: fill in real credentials/auth, region, and error handling.

```python
# PSEUDO-CODE (not executed)

import json

# 1) Create an SQS client
sdk = AwsSdk()
sqs = sdk.sqs(region="us-east-1")

# 2) Create (or look up) a queue (one-time setup)
resp = sqs.create_queue(
    QueueName="orders-queue",
    Attributes={
        "VisibilityTimeout": "60",      # seconds
        "MessageRetentionPeriod": "345600",  # 4 days
        # "RedrivePolicy": "{...}"      # configure a DLQ in real setups
    },
)
queue_url = resp["QueueUrl"]

# 3) Producer: send a message
sqs.send_message(
    QueueUrl=queue_url,
    MessageBody=json.dumps({"order_id": "o_123", "event": "created"}),
    # FIFO only:
    # MessageGroupId="orders",
    # MessageDeduplicationId="o_123-created",
)

# 4) Consumer: poll, process, then delete
while True:
    resp = sqs.receive_message(
        QueueUrl=queue_url,
        MaxNumberOfMessages=1,
        WaitTimeSeconds=20,  # long polling
        VisibilityTimeout=60,
    )

    messages = resp.get("Messages", [])
    if not messages:
        continue

    msg = messages[0]
    receipt = msg["ReceiptHandle"]
    payload = json.loads(msg["Body"])

    try:
        # IMPORTANT: make this idempotent (duplicates can occur)
        handle_order_event(payload)
        sqs.delete_message(QueueUrl=queue_url, ReceiptHandle=receipt)
    except Exception:
        # If you don't delete it, the message becomes visible again after the visibility timeout.
        # After N receives (configured), it can be moved to a DLQ.
        pass
```


## Practical notes

- Design consumers to be **idempotent** (Standard queues can deliver duplicates).
- Set **visibility timeout** longer than your worst-case processing time (or extend it while processing).
- Use a **DLQ** to avoid poison messages blocking progress.
- Consider **batching** (`SendMessageBatch` / `ReceiveMessage` with multiple messages) for throughput/cost.
